In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO

import json
from tqdm import tqdm_notebook as tqdm

In [2]:
url_search = 'https://rating.chgk.info/api/tournaments.json/search?name={}'
url_tournament = 'https://rating.chgk.info/api/tournaments/{}/list'
url_recaps = 'https://rating.chgk.info/api/tournaments/{idtournament}/recaps/{idteam}'
url_player = 'https://rating.chgk.info/api/players/{}'

In [79]:
# countries = ['Израиль','Германия','Россия','Украина','Беларусь','США','Великобритания','Молдова','Латвия','Литва','Эстония','Узбекистан','Казахстан','Таджикистан']
countries = ['Израиль','Германия','Россия','Беларусь','США','Великобритания','Молдова','Латвия','Литва','Эстония','Узбекистан','Казахстан','Таджикистан']

In [15]:
def get_tournament_list(query):
    response = requests.get(url_search.format(query)).text
    response = json.load(StringIO(response))
    return pd.DataFrame(response['items'])

In [57]:
def get_national_championships(countries):
    ts = []
    for country in countries:
        query = 'чемпионат '+ country[:-1]
        t = get_tournament_list(query)
        t = t[t.name.apply(lambda x: x.split()[-1][1:-1] == country[1:-1])]
        t = pd.DataFrame({'idtournament': t.idtournament, 'year': t.date_end.apply(lambda x: x[:4]), 'country': country})
        ts.append(t)
    return pd.concat(ts).reset_index()

In [97]:
def make_clickable(ids):
    ids = ids.split(', ')
    ids = ['<a href="https://rating.chgk.info/tournament/{}">{}</a>'.format(val, val) for val in ids]
    return ', '.join(ids)

In [90]:
championchips = get_national_championships(countries)

In [99]:
championchips_wide = pd.pivot_table(championchips.drop('index', axis=1), index='year', columns='country', values='idtournament', fill_value='', aggfunc=lambda x: ', '.join(x))
championchips_wide = championchips_wide.style.format(make_clickable)

In [111]:
championchips_wide

country,Беларусь,Великобритания,Германия,Израиль,Латвия,Литва,Молдова,Россия,США,Эстония
year,,,,,,,,,,
1997,,,,,,,,,1914,
1998,,,,716,,,,,1913,
1999,,,,1166,,,,,1036,
2000,1090,,,1056,,,,,1238,
2001,1088,,,1150,,,,1082,,
2002,1242,,,1270,,,,1224,1234,
2003,1393,,,1381,,,1344,1369,1359,
2004,23,,,21,35,,36,22,40,
2005,85,,,102,87,,75,76,77,88


In [110]:
with open("championships.html", "w", encoding='utf-8') as f:
    f.write(championchips_wide.render())